# Goal 

**Inputs:**

1. Take an input context text 
2. Take input sentence

**Backend:**

1. Convert input(1) to tokens 
2. wikpedia_topic_mode(tokens) -> likely_topics
3. wikpedia_topic_mode(likely_topics) -> pun_possible_words
4. pun.insert_pun( input(2), pun_possible_words ) -> new sentence

**Output:**

1. Print new sentence



In [1]:
# imports 

import pun
from gensim.models.ldamodel import LdaModel as Lda
from gensim import corpora

In [2]:
# Load the model

wiki_topicmodel = Lda.load('models/180918_wikipedia_model.200.gensim.')

# Load the corresponding dictionary 



In [ ]:
wiki_topicmodel.

In [3]:
test_inputs = {'wikipedia_climbing': "Climbing is the activity of using one's hands, feet, or any other part of the body to ascend a steep object. It is done for locomotion, recreation and competition, in trades that rely on it, and in emergency rescue and military operations. It is done indoors and out, on natural and man-made structures."}

In [4]:
# Define the context

context = 'apple banana cat tree plant'
# context = "I'm the king of england. I like to sweat and swim. I also enjoy cheese!"


# Tokenize context 

context_tokens = pun.tokenize(context)

# Convert to bow

bag_of_words = wiki_topicmodel.id2word.doc2bow(context_tokens)

In [5]:
test = wiki_topicmodel.get_document_topics(bag_of_words ) 

In [6]:
def get_words_from_top_topics(topic_list, model):
    
    list_of_words = []
    
    topic_list.sort(key=lambda tup: tup[1], reverse=True)
    
    for topic, topic_prob in topic_list:
        
        if topic_prob < 0.005:
            break
        
        for word_id, word_prob in model.get_topic_terms(topic, 500):
            if word_prob < 0.01:
                break
            list_of_words.append(model.id2word[word_id])
    
    return list_of_words

In [7]:
words = get_words_from_top_topics(test, wiki_topicmodel)

In [8]:
words

['species',
 'family',
 'plant',
 'grow',
 'animals',
 'animal',
 'genus',
 'flower',
 'forest',
 'small',
 'tree',
 'long',
 'fruit',
 'large',
 'america',
 'wild',
 'tail',
 'families',
 'gold',
 'class',
 'finish',
 'track',
 'race',
 'compete',
 'cat',
 'paralympics',
 'summer',
 'third',
 'apple',
 'inc',
 'second',
 'circuit',
 'disability',
 'type',
 'individual',
 'spain',
 'swimmer',
 'blind',
 'freestyle',
 'derby',
 'usually',
 'term',
 'type',
 'refer',
 'common',
 'sometimes',
 'kind',
 'describe',
 'similar',
 'generally',
 'example',
 'kinds',
 'case',
 'particular',
 'occur',
 'certain',
 'examples',
 'normally',
 'normal']

In [10]:
input_sentence = "There are people who can do and people who cant"

In [11]:
pun.insert_pun(input_sentence, words)

'There are people who cat do and people who cant'

NameError: name 'create_engine' is not defined

In [68]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [69]:
user = 'postgres' #add your username here (same as previous postgreSQL)
host = 'localhost'
dbname = 'birth_db'
db = create_engine('postgres://%s%s/%s'%(user,host,dbname))
con = None
con = psycopg2.connect(database = dbname, user = user)

OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/tmp/.s.PGSQL.5432"?


In [70]:
db

Engine(postgres://postgreslocalhost/birth_db)

In [71]:
import flaskexample

ModuleNotFoundError: No module named 'flaskexample'